### Installation of dependencies
We will use the HuggingFace library for this experiment.

In [ ]:
% pip install bitsandbytes datasets accelerate loralib
% pip install peft transformers tensorboard

### Loading the dataset
Here we load the dataset and create the train:test(80%:20%) split

In [2]:
from datasets import load_dataset

squad = load_dataset("squad", split="train[:5000]")
squad = squad.train_test_split(test_size=0.2)

In [ ]:
print(squad['train'][10])

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

#### Preprocessing
We need te preprocess the data before training, this means we will tokenize the context. We also need to find the locations of the answers inside the context.

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

def preprocess_train_data(examples, max_length=380, stride=128):
    # Tokenize the questions and context sequences
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
      questions,
      examples["context"],
      truncation="only_second",
      padding="max_length",
      stride=stride,
      max_length=max_length,
      return_offsets_mapping=True,
      return_overflowing_tokens=True,
    )

    answers = examples["answers"]
    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")

    start_positions = []
    end_positions = []

    # find the start and end positions of the answer within the context
    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # if the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions

    return inputs
processed_train_data = squad["train"].map(preprocess_train_data, batched=True, remove_columns=squad["train"].column_names)
processed_train_data

def preprocess_test_data(examples, max_length=380, stride=128):
    # Tokenize the questions and context sequences
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
      questions,
      examples["context"],
      truncation="only_second",
      padding="max_length",
      stride=stride,
      max_length=max_length,
      return_offsets_mapping=True,
      return_overflowing_tokens=True,
    )
    
    example_ids = []
    answers = examples["answers"]
    offset_mapping = inputs["offset_mapping"]
    sample_map = inputs.pop("overflow_to_sample_mapping")

    start_positions = []
    end_positions = []

    # find the start and end positions of the answer within the context
    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx])
        answer = answers[sample_idx]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)
        
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # if the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["example_id"] = example_ids  # keep the unique ID of the example
    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions

    return inputs
processed_test_data = squad["test"].map(preprocess_test_data, batched=True, remove_columns=squad["test"].column_names)
processed_test_data

In [ ]:
print(processed_test_data[0])

In [7]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

#### Loading of the original model

In [ ]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer
from peft import LoraConfig, TaskType, get_peft_model
from trl import SFTTrainer

model = AutoModelForQuestionAnswering.from_pretrained("distilbert/distilbert-base-uncased")

#### Setting training arguments
We set here different hyperaparameters like learning rate, batch sizes, number of epochs. We also create our Trainer instance.

In [ ]:
peft_config = LoraConfig(
                        r=8,
                        lora_alpha=32,
                        lora_dropout=0.01,
                        # target_modules= "all-linear",
                        target_modules= ["q_lin", "k_lin","v_lin"],
                        task_type=TaskType.QUESTION_ANS)

model = get_peft_model(model, peft_config)
model.merge_and_unload()
model.print_trainable_parameters()

training_args = TrainingArguments(
    output_dir="squad_lora_ft",
    eval_strategy="epoch",
    learning_rate=2e-2,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=processed_train_data,
    eval_dataset=processed_test_data,
    tokenizer=tokenizer,
    data_collator=data_collator,
)


#### Evaluation

In [11]:
import evaluate
import collections
import numpy as np
from tqdm import tqdm

def compute_metrics(start_logits, end_logits, features, examples, n_best=20, max_answer_length=50):
    metric = evaluate.load("squad")

    # keep a dictionary that maps examples to predictions through unique IDs
    example_to_features = collections.defaultdict(list)
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    predicted_answers = []
    for example in tqdm(examples):
        example_id = example["id"]
        context = example["context"]
        answers = []

        # loop through all features associated with that example
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            # keep a list of the top-k best predictions for the start and end position indexes
            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # skip answers with a length that is either < 0 or > max_answer_length
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    # reconstruct the answer considering each prediction for the start and end positions 
                    answer = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                    answers.append(answer)

        # select the answer with the best score based on the logit scores
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            # create a list with the predictions that contains the IDs and actual text
            # see: https://huggingface.co/spaces/evaluate-metric/squad
            predicted_answers.append(
                {"id": example_id, "prediction_text": best_answer["text"]}
            )
        else:
            predicted_answers.append({"id": example_id, "prediction_text": ""})

    # create a list with the labels that contains the IDs and actual text
    # see: https://huggingface.co/spaces/evaluate-metric/squad
    theoretical_answers = [{"id": ex["id"], "answers": ex["answers"]} for ex in examples]
    return metric.compute(predictions=predicted_answers, references=theoretical_answers)

In [ ]:
predictions, _, _ = trainer.predict(processed_test_data)
start_logits, end_logits = predictions
print("EM score and F1 before training: ")
compute_metrics(start_logits, end_logits, processed_test_data, squad["test"])

In [ ]:
trainer.train()

In [ ]:
predictions, _, _ = trainer.predict(processed_test_data)
start_logits, end_logits = predictions
print("EM score and F1 after training: ")
compute_metrics(start_logits, end_logits, processed_test_data, squad["test"])